In [1]:
import os
import glob
import numpy as np
from skimage.io import imread
import napari
import optireg

%matplotlib notebook

# Select image folder

In [2]:
master_folder = os.path.join('Y:',os.sep,'Kerim_Anlas','gastruloid_imaging','tracking_preprocess','preprocess')

In [3]:
# folder = os.path.join('Y:','/Kerim_Anlas/gastruloid_imaging/tracking_preprocess',
#                         'preprocess/20211110_ESLmix_48h_firstTex_probnochi/Pos3_best_reg_g')
# folder = os.path.join('Y:','/Kerim_Anlas/gastruloid_imaging/tracking_preprocess',
#                       'preprocess/2021_PD_72h/20210429_pos3_72h_reg_g')
folder = os.path.join(master_folder,'20220211_a2imix_96h_good','Pos1')

### PARAMETERS for registration
available_channels = ['mKO2', 'GFP'] # available channels (folders)
pixel_size = (2,0.3467,0.3467) # anistotropic pixel size (ordered: ZYX)
reference_channel = 'mKO2' # channel used to find cells
cell_diameter_XY = 10 # in pixels
reverse_order = True # process from last timepoint to first (i.e. register all images relative to last frame)
threshold = 200 # maxima with values lower than threshold will not be considered as cells
gpu_down = 1. # scale image down to use less GPU memory and improve speed
with_cle = True


# Find all tif files in the folder

In [4]:
flist = glob.glob(os.path.join(folder, reference_channel, '*.tif'))
flist.sort()
# flist = flist[:10]
print('Found %d images.'%len(flist))

Found 175 images.


# Find anchor points

In [5]:
scale = pixel_size[0]/pixel_size[1]
print('Pixel size in Z : %.5f um'%pixel_size[0])
print('Pixel size in XY: %.5f um'%pixel_size[1])
print('Anisotropy: %.5f'%scale)

Pixel size in Z : 2.00000 um
Pixel size in XY: 0.34670 um
Anisotropy: 5.76868


In [6]:
# compute anchor points or load file if it already exists
df = optireg.find_anchor_points(
    folder, flist, fileName = 'anchor_points.csv', 
    scale = scale, gpu_down = gpu_down, 
    cell_diameter_XY = cell_diameter_XY,
    reverse_order = reverse_order,
    threshold = threshold
)


  0%|                                                                                          | 0/175 [00:01<?, ?it/s]


LogicError: clCreateImage failed: INVALID_OPERATION

In [ ]:
df

# Optional: Visualize anchor points

In [ ]:
start = 0
stop = 22
visualize_images = True

#######################################
if visualize_images:
    img = np.array([imread(f) for f in flist[start:stop]])

df_napari = df[(df.tp>=start)&(df.tp<stop)][['tp','z','y','x']].to_numpy()
df_napari[:,0] -= start

viewer = napari.Viewer()
if visualize_images:
    viewer.add_image(img, scale=(scale,1,1), name='gastruloid')
viewer.add_points(df_napari, name='cells', opacity=1., 
                  face_color='red')
napari.run()


# method: minimize average distance to closest point

In [ ]:
tps = list(set(df.tpreg))
tps.sort()
df_list = [df[df.tpreg==tp] for tp in tps]

Ts = optireg.find_transformations(df_list, folder, maxiter=100, tol=1e-5)

In [ ]:
df_list = optireg.perform_points_registration(df_list, Ts)

# Optional: Visualize anchor points

In [ ]:
import napari

df_napari = []
for df_tp in df_list:
    poss = df_tp[['tp','z','y','x']].to_numpy()
    for p in poss:
        df_napari.append(p)
df_napari = np.array(df_napari)

df_napari1 = []
for df_tp in df_list:
    poss = df_tp[['tp','zreg','yreg','xreg']].to_numpy()
    for p in poss:
        df_napari1.append(p)
df_napari1 = np.array(df_napari1)

viewer = napari.Viewer()
viewer.add_points(df_napari, name='cells', opacity=1, 
#                   scale=(1,1,1),
                  face_color='red')
viewer.add_points(df_napari1, name='cells_reg', opacity=1, 
#                   scale=(1,1,1),
                  face_color='lightgreen')
napari.run()


# Perform image registration

In [ ]:
from importlib import reload

reload(optireg)
optireg.perform_images_registration(folder, 
                                    available_channels, 
                                    reverse_order, 
                                    scale,
                                    Ts,
                                    with_cle = True,
                                    registered_folder_id = '_reg1')

In [ ]:
# import subprocess as sp

# def get_gpu_memory():
#     command = "nvidia-smi --query-gpu=memory.free --format=csv"
#     memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
#     memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
#     return memory_free_values

# get_gpu_memory()